In [1]:
%matplotlib inline
import requests
import json
import ast
import string
import pandas as pd
import pymongo
from random import randint
import re
import numpy as np
import matplotlib.pyplot as plt
from datetime import date

In [2]:
def send_request(jobCate):
    try:
        response = requests.get( url="https://www.amazon.jobs/en/search.json?",params={
                "base_query": "",
                "category[]": jobCate,
                "city": "",
                "country": "",
                "county": "",
                "facets[]": "location,category,normalized_location,job_function_id, business_category, schedule_type_id, employee_class, normalized_location, job_function_id",
                "latitude": "",
                "loc_group_id": "",
                "loc_query": "",
                "longitude": "",
                "offset": 0,
                "query_options": "",
                "radius": "24km",
                "region": "",
                "result_limit": 10000,
                "sort": "relevant"},
            headers={
                "Host": "www.amazon.jobs",
                "Connection": "keep-alive",
                "Pragma": "no-cache",
                "Cache-Control": "no-cache",
                "Accept": "*/*",
                "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/78.0.3904.70 Safari/537.36",
                "Accept-Encoding": "gzip, deflate",
                "Cookie": "preferred_locale=en-US"})
        print('Response HTTP Status Code: {status_code}'.format(
            status_code=response.status_code))
        return response.text
    except requests.exceptions.RequestException:
        print('HTTP Request failed')

In [3]:
def geoL(city_L, country_L, cities, state_L, state, lat, lng, country_code):#, Lat_Lon):
    if country_L == "US":
        for count2 in range(0,len(cities)): 
            
            if city_L == cities[count2] and state_L == state[count2]:
                Lat_Lon = [lat[count2],lng[count2]]
                break
            if count2 == len(cities)-1:
                Lat_Lon = ["",""] 
    else:
        for count2 in range(0,len(cities)):
            if city_L == cities[count2]:
                Lat_Lon = [lat[count2],lng[count2]]
                break
            if count2 == len(cities)-1:
                for count in range(0,len(cities)):
                    if country_code[count] == country_L:
                        Lat_Lon = [lat[count],lng[count]]
                        break
                    if count == len(cities)-1:
                        Lat_Lon = ["",""]
    return Lat_Lon

In [4]:
#Get geocoordinates from a file
geocoor = pd.read_csv("worldcities.csv")
cities = geocoor["city_ascii"].tolist()
country_code = geocoor["iso2"].tolist()
state = geocoor["admin_name"].tolist()
lat = geocoor["lat"].tolist()
lng = geocoor["lng"].tolist()

In [5]:
def addr(Location):
    try:
        comP_loc = Location.split(", ")
        if (len(comP_loc) == 3):
            st = comP_loc[1]
        if (len(comP_loc) == 2):
            st = "NaN"
        return st
    except:
        st = "NaN"
        return st

In [6]:
def updating(job_categories):
    category_add_rem = []    
    jobs_L = []
    job_list_to_add = []
    job_list_to_rem = []

    for job_cat in job_categories:
        raw_D = send_request(job_cat)    #aqui llamando a la funcion
        raw_D = json.loads(raw_D)
        jobs = raw_D["jobs"]             #esta lista contiene todos los roles que van a ser documents en el collection

        jobs_dic = {}
        city_L = []
        country_L =[]
        state_L = []
        job_ID_list = []
        arr = []
        
        for job in jobs:
            jobs_dic["category"] = job["job_category"]
            jobs_dic["title"] = job["title"]
            jobs_dic["Job_ID"] = job["id_icims"]
            jobs_dic["City"] = job["city"]
            jobs_dic["Country"] = job["country_code"]

            statex = job["normalized_location"]
            state_L = addr(statex)           #aqui llamo a la funcion ADDR
            
            jobs_dic["Posted_date"] = job["posted_date"]
            jobs_dic["Description"] = job["description"]
            jobs_dic["Basic_Qualif"] = job["basic_qualifications"]
            jobs_dic["Preferred_Qualif"] = job["preferred_qualifications"]
            jobs_dic["Company"] = job["company_name"]
            jobs_dic["Apply"] = job["url_next_step"]
            jobs_dic["Geocoordinates"] = geoL(job["city"], job["country_code"], cities, state_L, state, lat, lng, country_code)
            job_ID_list.append(job["id_icims"])
            jobs_L.append(jobs_dic.copy())

        for obj in collection.find():
            try:
                formaObj = obj["category"].lower()
                formaObj = re.sub("/", " ", formaObj)
                formaObj = re.sub("-", " ", formaObj)
                formaObj = formaObj.translate(str.maketrans('', '', string.punctuation))
                formaObj = re.sub(" +", " ", formaObj)
                formaObj = re.sub(" ", "-", formaObj)
            except:
                print("error", obj["Job_ID"])

            if formaObj == job_cat:
                arr.append(obj["Job_ID"])

        new_jobs = list(set(job_ID_list) - set(arr))
        remove_jobs = list(set(arr) - set(job_ID_list))
        job_list_to_add = job_list_to_add + new_jobs
        job_list_to_rem = job_list_to_rem + remove_jobs
        category_add_rem.append([len(new_jobs),len(remove_jobs)])
    return category_add_rem, job_list_to_add, job_list_to_rem, jobs_L

In [7]:
job_categories = ["administrative-support",                    #01done
                  "audio-video-photography-production",        #02done
                 "business-merchant-development",              #03done
                 "business-intelligence",                      #04done
                 "buying-planning-instock-management",         #05done
                 "customer-service",                           #06done
                 "data-science",                               #07done
                 "database-administration",                    #08done
                 "design",                                     #09done
                 "economics",                                  #10done
                 "editorial-writing-content-management",       #11done
                 "facilities-maintenance-real-estate",         #12done
                 "finance-accounting",                         #13done
                 "fulfillment-operations-management",
                 "fulfillment-warehouse-associate",
                 "hardware-development",
                 "human-resources",                            #17done
                 "investigation-loss-prevention",
                 "leadership-development-training",
                 "legal",
                 "machine-learning-science",                   #21done
                 "marketing-pr",
                 "medical-health-safety",
                 "operations-it-support-engineering",          #24done
                 "project-program-product-management-non-tech",#25done
                 "public-policy",
                 "research-science",                           #27done
                 "sales-advertising-account-management",
                 "software-development",                      #29done
                 "solutions-architect",                        #30done
                 "supply-chain-transportation-management",     #31done
                 "systems-quality-security-engineering",       #32done
                 "project-program-product-management-technical"] #33done

In [8]:
conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn)

db = client.jobs
collection = db.roles
collection2 = db.filled

In [9]:
Jobs_L = updating(job_categories)

Response HTTP Status Code: 200
Response HTTP Status Code: 200
Response HTTP Status Code: 200
Response HTTP Status Code: 200
Response HTTP Status Code: 200
Response HTTP Status Code: 200
Response HTTP Status Code: 200
Response HTTP Status Code: 200
Response HTTP Status Code: 200
Response HTTP Status Code: 200
Response HTTP Status Code: 200
Response HTTP Status Code: 200
Response HTTP Status Code: 200
Response HTTP Status Code: 200
Response HTTP Status Code: 200
Response HTTP Status Code: 200
Response HTTP Status Code: 200
Response HTTP Status Code: 200
Response HTTP Status Code: 200
Response HTTP Status Code: 200
Response HTTP Status Code: 200
Response HTTP Status Code: 200
Response HTTP Status Code: 200
Response HTTP Status Code: 200
Response HTTP Status Code: 200
Response HTTP Status Code: 200
Response HTTP Status Code: 200
Response HTTP Status Code: 200
Response HTTP Status Code: 200
Response HTTP Status Code: 200
Response HTTP Status Code: 200
Response HTTP Status Code: 200
Response

In [10]:
for new_job in Jobs_L[1]:
    for job in Jobs_L[3]:
        if new_job == job["Job_ID"]:
            collection.insert_one(job)
for filled_job in Jobs_L[2]:
    collection.delete_one({"Job_ID" : filled_job})
            

In [11]:
today = date.today()
jobs_filled = {}
jobs_F = []
for j in range(0,len(Jobs_L[0])):
    jobs_filled["category"] = job_categories[j]
    jobs_filled["#_Jobs_added"] = Jobs_L[0][j][0]
    jobs_filled["#_Jobs_filled"] = Jobs_L[0][j][1]
    jobs_filled["Date"] = today.strftime("%m/%d/%y")
    jobs_F.append(jobs_filled.copy())
    
collection2.insert(jobs_F)

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:11: DeprecationWarning: insert is deprecated. Use insert_one or insert_many instead.
  # This is added back by InteractiveShellApp.init_path()


[ObjectId('5dfd5fd10d9a665c5b11adeb'),
 ObjectId('5dfd5fd10d9a665c5b11adec'),
 ObjectId('5dfd5fd10d9a665c5b11aded'),
 ObjectId('5dfd5fd10d9a665c5b11adee'),
 ObjectId('5dfd5fd10d9a665c5b11adef'),
 ObjectId('5dfd5fd10d9a665c5b11adf0'),
 ObjectId('5dfd5fd10d9a665c5b11adf1'),
 ObjectId('5dfd5fd10d9a665c5b11adf2'),
 ObjectId('5dfd5fd10d9a665c5b11adf3'),
 ObjectId('5dfd5fd10d9a665c5b11adf4'),
 ObjectId('5dfd5fd10d9a665c5b11adf5'),
 ObjectId('5dfd5fd10d9a665c5b11adf6'),
 ObjectId('5dfd5fd10d9a665c5b11adf7'),
 ObjectId('5dfd5fd10d9a665c5b11adf8'),
 ObjectId('5dfd5fd10d9a665c5b11adf9'),
 ObjectId('5dfd5fd10d9a665c5b11adfa'),
 ObjectId('5dfd5fd10d9a665c5b11adfb'),
 ObjectId('5dfd5fd10d9a665c5b11adfc'),
 ObjectId('5dfd5fd10d9a665c5b11adfd'),
 ObjectId('5dfd5fd10d9a665c5b11adfe'),
 ObjectId('5dfd5fd10d9a665c5b11adff'),
 ObjectId('5dfd5fd10d9a665c5b11ae00'),
 ObjectId('5dfd5fd10d9a665c5b11ae01'),
 ObjectId('5dfd5fd10d9a665c5b11ae02'),
 ObjectId('5dfd5fd10d9a665c5b11ae03'),
 ObjectId('5dfd5fd10d9a66

In [ ]:
collection.update({},{"$unset": {"Apply":" "}},{"multi": True});

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: DeprecationWarning: update is deprecated. Use replace_one, update_one or update_many instead.
  """Entry point for launching an IPython kernel.
